# 

# CCDI Hub Extract Samples from Reference Files

This notebook cleans the Excel file from C3DC Hub and extracts samples from the MCI and RMS studies reference files from both CCDI and C3DC Hub

Author: Lucy Han, Booz Allen Hamilton

In [1]:
# import pandas and os
import pandas as pd
import os

In [2]:
# checking current path for Jupyter Notebook
current_path = os.getcwd()
print(current_path)

/sbgenomics/workspace


In [3]:
# src path
src = "/sbgenomics/project-files/Reference_Files"

In [4]:
# output path
output = "/sbgenomics/output-files"

In [5]:
# confirm directory
os.chdir(src)

In [6]:
# list the files in the directory
!ls

 C3DC_phs000720_CCDI_RMS-Mut-Pred_Study_Manifest_v2.0.0.xlsx
'C3DC_phs002790_CCDI MCI_Study_Manifest_v2.1.0.xlsx'
'CCDI Hub_CCDI MCI_RMS Pts_Participants Download 2025-08-25 12-50-30.csv'
'CCDI Hub_CCDI MCI_RMS_Samples Download 2025-08-25 13-24-08.csv'
'CCDI Hub_MCI Path DICOM Files Manifest_2025-09-18 21-12-26.csv'
'CCDI Hub_RMS-Mutation-Prediction_phs000720_Participants_Download 2025-07-18 16-11-54.csv'
'CCDI Hub_RMS-Mutation-Prediction_Samples Download 2025-08-25 16-10-57.csv'
'CCDI Hub_RMS Path DICOM Files Manifest_2025-09-18 21-24-21.csv'
 Filtered
 img_rand_sample_phs002790_phs000720.csv


## Read in reference files

In [7]:
# MCI Reference File names
MCI_study_file = 'C3DC_phs002790_CCDI MCI_Study_Manifest_v2.1.0.xlsx'
MCI_part_file = 'CCDI Hub_CCDI MCI_RMS Pts_Participants Download 2025-08-25 12-50-30.csv'
MCI_samples_file = 'CCDI Hub_CCDI MCI_RMS_Samples Download 2025-08-25 13-24-08.csv'
MCI_dicom_file = 'CCDI Hub_MCI Path DICOM Files Manifest_2025-09-18 21-12-26.csv'

# RMS Reference File names
RMS_study_file = 'C3DC_phs000720_CCDI_RMS-Mut-Pred_Study_Manifest_v2.0.0.xlsx'
RMS_part_file = 'CCDI Hub_RMS-Mutation-Prediction_phs000720_Participants_Download 2025-07-18 16-11-54.csv'
RMS_samples_file = 'CCDI Hub_RMS-Mutation-Prediction_Samples Download 2025-08-25 16-10-57.csv'
RMS_dicom_file = 'CCDI Hub_RMS Path DICOM Files Manifest_2025-09-18 21-24-21.csv'

# Image sample file (sample was generated outside of the SB-CGC environment)
image_sample_file = 'img_rand_sample_phs002790_phs000720.csv'

In [8]:
# Read MCI files
MCI_part_ref_df = pd.read_csv(MCI_part_file)
MCI_samples_ref_df = pd.read_csv(MCI_samples_file)
MCI_dicom_ref_df = pd.read_csv(MCI_dicom_file)

# Read RMS files
RMS_part_ref_df = pd.read_csv(RMS_part_file)
RMS_samples_ref_df = pd.read_csv(RMS_samples_file)
RMS_dicom_ref_df = pd.read_csv(RMS_dicom_file)

# read image sample
sample_ref_df = pd.read_csv(image_sample_file)

In [ ]:
# check shape of the dataframe
sample_ref_df.shape

In [11]:
# Read MCI Excel study file
MCI_xl = pd.ExcelFile(MCI_study_file)

In [12]:
# Read RMS Excel study file
RMS_xl = pd.ExcelFile(RMS_study_file)

## Process Excel files from C3DC Hub

The Excel files have many tabs. This section filters out the tabs in the Excel file that do not have a participant.participant_id column and returns a new Excel sheet with only the tabs that have participant.participant_id that have been selected in the sample.

In [13]:
def process_excel_files(input_xl_file):
    # function to process excel files
    
    # input_xl_file is the file path to the excel file to read in
    
    input_excel_file = input_xl_file # input excel file path
    output_excel_file = f"{output}/filtered_{input_xl_file}" # Output excel file path
    ref_identifier_column_name = 'Participant ID'  # Column name for identifiers in both the DataFrame and Excel sheets
    identifier_column_name = 'participant.participant_id'
    
    identifier_list = sample_ref_df[ref_identifier_column_name].astype(str).tolist()
    
    # load excel file
    xlsx = pd.ExcelFile(input_excel_file)
    sheet_names = xlsx.sheet_names
    
    # Use ExcelWriter to outp
    with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
        for sheet in sheet_names:
            df = pd.read_excel(input_excel_file, sheet_name=sheet)
            
            # Skip sheet if identifier column doesn't exist
            if identifier_column_name not in df.columns:
                print(f"Skipping sheet '{sheet}' - no '{identifier_column_name}' column found.")
                continue
            
            # Ensure identifier column is string type
            df[identifier_column_name] = df[identifier_column_name].astype(str)
    
            # Filter based on identifiers
            filtered_df = df[df[identifier_column_name].isin(identifier_list)]
    
            # Write filtered data to the same-named sheet in the output file
            filtered_df.to_excel(writer, sheet_name=sheet, index=False)
    
    print(f"Filtered Excel saved to: {output_excel_file}")
    return output_excel_file

In [14]:
filtered_MCI_study_file = process_excel_files(MCI_study_file)

filtered_MCI_study_xl = pd.ExcelFile(filtered_MCI_study_file)

Skipping sheet 'README and INSTRUCTIONS' - no 'participant.participant_id' column found.
Skipping sheet 'study' - no 'participant.participant_id' column found.
Skipping sheet 'study_admin' - no 'participant.participant_id' column found.
Skipping sheet 'study_arm' - no 'participant.participant_id' column found.
Skipping sheet 'study_funding' - no 'participant.participant_id' column found.
Skipping sheet 'study_personnel' - no 'participant.participant_id' column found.
Skipping sheet 'publication' - no 'participant.participant_id' column found.
Skipping sheet 'participant' - no 'participant.participant_id' column found.
Skipping sheet 'cell_line' - no 'participant.participant_id' column found.
Skipping sheet 'pdx' - no 'participant.participant_id' column found.
Skipping sheet 'sequencing_file' - no 'participant.participant_id' column found.
Skipping sheet 'methylation_array_file' - no 'participant.participant_id' column found.
Skipping sheet 'cytogenomic_file' - no 'participant.participa

In [15]:
filtered_MCI_study_xl.sheet_names

['diagnosis',
 'survival',
 'treatment',
 'treatment_response',
 'synonym',
 'family_relationship',
 'medical_history',
 'exposure',
 'radiology_file',
 'molecular_test',
 'sample_RMS',
 'sample',
 'clinical_measure_file',
 'generic_file']

In [16]:
filtered_RMS_study_file = process_excel_files(RMS_study_file)

filtered_RMS_study_xl = pd.ExcelFile(filtered_RMS_study_file)

Skipping sheet 'README and INSTRUCTIONS' - no 'participant.participant_id' column found.
Skipping sheet 'study' - no 'participant.participant_id' column found.
Skipping sheet 'study_admin' - no 'participant.participant_id' column found.
Skipping sheet 'study_arm' - no 'participant.participant_id' column found.
Skipping sheet 'study_funding' - no 'participant.participant_id' column found.
Skipping sheet 'study_personnel' - no 'participant.participant_id' column found.
Skipping sheet 'publication' - no 'participant.participant_id' column found.
Skipping sheet 'participant' - no 'participant.participant_id' column found.
Skipping sheet 'cell_line' - no 'participant.participant_id' column found.
Skipping sheet 'pdx' - no 'participant.participant_id' column found.
Skipping sheet 'sequencing_file' - no 'participant.participant_id' column found.
Skipping sheet 'methylation_array_file' - no 'participant.participant_id' column found.
Skipping sheet 'cytogenomic_file' - no 'participant.participa

In [17]:
filtered_RMS_study_xl.sheet_names

['diagnosis',
 'survival',
 'treatment',
 'treatment_response',
 'synonym',
 'family_relationship',
 'medical_history',
 'exposure',
 'radiology_file',
 'molecular_test',
 'sample',
 'clinical_measure_file',
 'generic_file']

## Process csv files

This section filters the csv files to only include data that have been selected in the sample.

In [41]:
def process_csv_files(input_file, identifier_column_name = 'Participant ID'):
    # function to process csv files
    
    # input_file is the file path to the csv file to read in
    # identifier_column_name is the id_column with the default as 'Participant ID'
    
    id_column_name = 'Participant ID'
    input_csv_file = input_file      # Source CSV file
    output_csv_file = f"{output}/filtered_{input_csv_file}"   # Output CSV file
    
    # load source data
    source_df = pd.read_csv(input_csv_file)
    
    # load identifiers dataframe
    # Replace with your real identifier list or load from a file
    identifiers_df = sample_ref_df
    
    # filter rows
    # Ensure both columns are of string type for reliable matching
    source_df[identifier_column_name] = source_df[identifier_column_name].astype(str)
    identifiers_df[id_column_name] = identifiers_df[id_column_name].astype(str)
    
    # Filter source data where IDs are in identifiers list
    filtered_df = source_df[source_df[identifier_column_name].isin(identifiers_df[id_column_name])]
    
    # save dataframe
    filtered_df.to_csv(output_csv_file, index=False)
    
    print(f"Filtered data saved to: {output_csv_file}")
    return output_csv_file

#### Process MCI CSV Files

In [42]:
filtered_MCI_part_file = process_csv_files(MCI_part_file)
filtered_MCI_part_df = pd.read_csv(filtered_MCI_part_file)

filtered_MCI_part_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_CCDI MCI_RMS Pts_Participants Download 2025-08-25 12-50-30.csv


(71, 9)

In [30]:
filtered_MCI_samples_file = process_csv_files(MCI_samples_file)
filtered_MCI_samples_df = pd.read_csv(filtered_MCI_samples_file)

filtered_MCI_samples_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_CCDI MCI_RMS_Samples Download 2025-08-25 13-24-08.csv


(163, 8)

In [31]:
filtered_MCI_dicom_file = process_csv_files(MCI_dicom_file)
filtered_MCI_dicom_df = pd.read_csv(filtered_MCI_dicom_file)

filtered_MCI_dicom_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_MCI Path DICOM Files Manifest_2025-09-18 21-12-26.csv


(462, 10)

#### Process RMS CSV Files

In [32]:
filtered_RMS_part_file = process_csv_files(RMS_part_file)
filtered_RMS_part_df = pd.read_csv(filtered_RMS_part_file)

filtered_RMS_part_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_RMS-Mutation-Prediction_phs000720_Participants_Download 2025-07-18 16-11-54.csv


(34, 9)

In [33]:
filtered_RMS_samples_file = process_csv_files(RMS_samples_file)
filtered_RMS_samples_df = pd.read_csv(filtered_RMS_samples_file)

filtered_RMS_samples_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_RMS-Mutation-Prediction_Samples Download 2025-08-25 16-10-57.csv


(34, 8)

In [43]:
filtered_RMS_dicom_file = process_csv_files(RMS_dicom_file, identifier_column_name = 'Sample ID')
filtered_RMS_dicom_df = pd.read_csv(filtered_RMS_dicom_file)

filtered_RMS_dicom_df.shape

Filtered data saved to: /sbgenomics/output-files/filtered_CCDI Hub_RMS Path DICOM Files Manifest_2025-09-18 21-24-21.csv


(246, 9)

In [44]:
!ls

 C3DC_phs000720_CCDI_RMS-Mut-Pred_Study_Manifest_v2.0.0.xlsx
'C3DC_phs002790_CCDI MCI_Study_Manifest_v2.1.0.xlsx'
'CCDI Hub_CCDI MCI_RMS Pts_Participants Download 2025-08-25 12-50-30.csv'
'CCDI Hub_CCDI MCI_RMS_Samples Download 2025-08-25 13-24-08.csv'
'CCDI Hub_MCI Path DICOM Files Manifest_2025-09-18 21-12-26.csv'
'CCDI Hub_RMS-Mutation-Prediction_phs000720_Participants_Download 2025-07-18 16-11-54.csv'
'CCDI Hub_RMS-Mutation-Prediction_Samples Download 2025-08-25 16-10-57.csv'
'CCDI Hub_RMS Path DICOM Files Manifest_2025-09-18 21-24-21.csv'
 img_rand_sample_phs002790_phs000720.csv
